# Download, extract and store data locally

# User params

In [1]:
# Possible categories to download:
# --------------------------------
# All_Beauty
# Amazon_Fashion
# Appliances
# Arts_Crafts_and_Sewing
# Automotive
# Baby_Products
# Beauty_and_Personal_Care
# Books
# CDs_and_Vinyl
# Cell_Phones_and_Accessories
# Clothing_Shoes_and_Jewelry
# Digital_Music
# Electronics
# Gift_Cards
# Grocery_and_Gourmet_Food
# Handmade_Products
# Health_and_Household
# Health_and_Personal_Care
# Home_and_Kitchen
# Industrial_and_Scientific
# Kindle_Store
# Magazine_Subscriptions
# Movies_and_TV
# Musical_Instruments
# Office_Products
# Patio_Lawn_and_Garden
# Pet_Supplies
# Software
# Sports_and_Outdoors
# Subscription_Boxes
# Tools_and_Home_Improvement
# Toys_and_Games
# Video_Games

category_to_download = "Gift_Cards"


# For dataset library
# ---------------------
# type of data to download
# entire unprocessed data: 
# - 'raw_'
# processed data : 
# - "0core_rating_", "0core_timestamp_", "0core_timestamp_w_his_"
# - "5core_rating_", "5core_timestamp_", "5core_timestamp_w_his_"
type = 'raw_'
split = "full" # "full", "train", "valid", "test"


# Utility Functions

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../src'))

# other imports
from pathlib import Path

# Processing - native

**Download**

In [ ]:
from src.utils.io.download import download_data
download_data(category_to_download)

**Merging**

In [ ]:
import os
from src.config import BASE_PATH_DATA
from src.utils.io.io import load_dataframe

# load data
file_name_review = category_to_download + "_review.json"
file_name_metadata = category_to_download + "_metadata.json"

# read json file into dataframe
df_metadata = load_dataframe(os.path.join(BASE_PATH_DATA, 'raw', file_name_review))
df_review = load_dataframe(os.path.join(BASE_PATH_DATA, 'raw', file_name_metadata))


# Merge the datasets on 'parent_asin' with suffixes for duplicate columns
merged_df = pd.merge(df_review, df_metadata, 
                     on="parent_asin", how="inner", suffixes=("_review", "_metadata"))

# Processing - using datasets library

In [4]:
from datasets import load_dataset

dataset_reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_" + category_to_download, trust_remote_code=True)
print(dataset_reviews[split].num_rows)
print(dataset_reviews[split][0])

dataset_medata = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_"+ category_to_download, split=split, trust_remote_code=True)
print(dataset_medata.num_rows)
print(dataset_medata[0])

Generating full split: 0 examples [00:00, ? examples/s]

152410
{'rating': 5.0, 'title': 'Great gift', 'text': 'Having Amazon money is always good.', 'images': [], 'asin': 'B00IX1I3G6', 'parent_asin': 'B00IX1I3G6', 'user_id': 'AHZ6XMOLEWA67S3TX7IWEXXGWSOA', 'timestamp': 1549866158332, 'helpful_vote': 0, 'verified_purchase': True}


Generating full split: 0 examples [00:00, ? examples/s]

1137
{'main_category': 'Gift Cards', 'title': 'Amazon.com Gift Card in Gift Tag (Various Designs)', 'average_rating': 4.8, 'rating_number': 1006, 'features': ['Gift Card is affixed inside a gift tag', 'Gift amount may not be printed on Gift Cards', 'Gift Card has no fees and no expiration date', 'No returns and no refunds on Gift Cards', 'Gift Card is redeemable towards millions of items storewide at Amazon.com', 'Scan and redeem any Gift Card with a mobile or tablet device via the Amazon App', 'Free One-Day Shipping (where available)', 'Customized gift message, if chosen at check-out, only appears on packing slip and not on the actual gift card or carrier'], 'description': ["Amazon.com Gift Cards are the perfect way to give them exactly what they're hoping for - even if you don't know what it is. Amazon.com Gift Cards are redeemable for millions of items across Amazon.com. Item delivered is a single physical Amazon.com Gift Card nested inside or with a free gift accessory."], 'price':

In [5]:
import pandas as pd
import os

# Tranform to pandas
df_review = pd.DataFrame(dataset_reviews[split])
df_metadata = pd.DataFrame(dataset_medata)

# merge
merged_df = pd.merge(df_review, df_metadata, on="parent_asin", how="inner", suffixes=("_review", "_metadata"))



# Save it

In [ ]:
dest_folder = Path("../data/raw")
save_name = os.path.join(dest_folder, f'merged_dataset.parquet')
merged_df.to_parquet(save_name, index=False)